In [7]:
import os
import textwrap
import json
import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown


import cx_Oracle

from datetime import datetime

import PIL.Image

In [17]:
# the path of input 
dirpath = r"\\10.25.25.216\FidelisFileServer\04數據與分析部\部門資料\LLM資料\華紙-營運績效分析月報\masked"

topic_name = "華紙-營運績效分析月報"

# prompt = "請用中文說明報表包含哪些指標分析內容"


prompt = "請用中文說明報表包含哪些指標分析內容，並且不要讀取裡面的數據內容只要專注在坐標軸就好"

### Helper functions

In [18]:
def read_json(data_path):
    with open(data_path) as f:
        data = json.load(f)
    return data

In [10]:
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

## Generate description for dashboard

In [12]:
oracle_client_dir = './opt/oracle/'

oracle_client_path = os.path.join(oracle_client_dir, os.listdir(oracle_client_dir)[0])

cx_Oracle.init_oracle_client(lib_dir=oracle_client_path)

ProgrammingError: Oracle Client library has already been initialized

In [22]:
info_path = '.env/connections.json'
info = read_json(info_path)
API_KEY= info['gkey']
genai.configure(api_key=API_KEY)

model = genai.GenerativeModel('gemini-1.5-flash')


In [7]:
def _insert_data(table_name, topic_name,topic_pic_name,topic_content,now ):
    sqlstat = "insert into " + table_name + " ( topic_name, topic_pic_name, topic_content, creation_date) values ( :topic_name,:topic_pic_name,:topic_content, :now )"
    try:
        db_info  = info['dw_conn']
        user = db_info['user']
        pw = db_info['pw']
        IP = db_info['IP']
        port = db_info['port']
        service_name = db_info['service_name']

        # establish a new connection
        dsn_tns = cx_Oracle.makedsn(IP, port, service_name=service_name)
        DWH_conn = cx_Oracle.connect(user=user, password=pw, dsn=dsn_tns)

        with DWH_conn as connection:
            # create a cursor
            with connection.cursor() as cursor:
                # execute the insert statement
                cursor.execute(sqlstat, [topic_name, topic_pic_name, topic_content, now])
                # commit work
                connection.commit()
    except cx_Oracle.Error as error:
            print('Error occurred:')
            print(error)

In [8]:
# dirpath = r"\\10.25.25.216\FidelisFileServer\04數據與分析部\部門資料\LLM資料\華紙-營運績效分析月報\masked"
now = datetime.now()
d1 = os.listdir(dirpath)

In [12]:
# dirpath = r"\\10.25.25.216\FidelisFileServer\04數據與分析部\部門資料\LLM資料\華紙-營運績效分析月報\masked"
now = datetime.now()
d1 = os.listdir(dirpath)
table_name = "ai_report"
for i  in d1[:2]:
    print(i)
    if i != "Thumbs.db" :
        filename = dirpath + "\\" + i
        img = PIL.Image.open(filename)
        response = model.generate_content([prompt, img], stream=True)
        response.resolve()
        topic_content = response.text
        topic_pic_name = filename
        print(topic_content)
        # _insert_data( table_name,topic_name,topic_pic_name,topic_content,now)

Thumbs.db
事業部層-全廠損益.jpg
報表包含了以下指標分析內容：

**本月：**

* 銷售收入
* 銷售成本
* 銷售毛利
* 營業利益
* 稅前淨益

**全年累計：**

* 銷售收入
* 銷售成本
* 銷售毛利
* 營業利益
* 稅前淨益
* 年度預算
* 達成率

**商品別：**

* 啤酒 (TWD)
* 久釀 (TWD)
* 花雕 (TWD)
* 台啤 (TWD)
* 啤酒 (TWD)
* P廠 (TWD)
* 啤酒 (CNY)
* 啤酒 (CNY)
* 啤酒 (CNY)
* 台啤 (TWD)
